In [1]:
import sys
sys.path.append('../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


# Simple control

In [2]:
RealWorld = ry.Config()
RealWorld.addFile("../scenarios/challenge.g")
obj = RealWorld.addFrame("object")
pos_obj = [0,0,.9]
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,1,0])
d=0.1
obj.setShape(ry.ST.ssBox, size=[d,d,d,d])
obj.setColor([1,0,0])
obj.setContact(1)
#obj.setColor([0,0,1])
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)

S.addSensor("camera")
camera = RealWorld.frame("camera")
C = ry.Config()
C.addFile('../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

low_threshold = np.array([30,150,50])
upp_threshold = np.array([255,255,180])

In [3]:
tau = .01
t = 0
step = 0.05
while True:
    time.sleep(0.01)
    t+=1
    q = S.get_q()
    S.step([], tau, ry.ControlMode.none)
    [y,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])
    camera.setPosition(y)
    [y,J] = RealWorld.evalFeature(ry.FS.quaternion, ["L_gripperCenter"])
    camera.setQuaternion(y)
    
    [y,J] = RealWorld.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "object"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y)
    S.step(vel, tau, ry.ControlMode.velocity)
    
    if t%5 == 0:
        [rgb, depth] = S.getImageAndDepth()
        binary = cv.inRange(cv.cvtColor(rgb, cv.COLOR_BGR2HSV), low_threshold, upp_threshold) #'hue','saturation','value'.
        #ret, binary = cv.threshold(cv.cvtColor(img, cv.COLOR_BGR2GRAY),
        #                127, 255, cv.THRESH_BINARY)
        contours, hier = cv.findContours(binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            (x, y), radius = cv.minEnclosingCircle(contours[0])
            for c in contours:
                (x1, y1), r = cv.minEnclosingCircle(c)
                if r>radius:
                    (x,y)=(x1, y1)
                    radius = r
            cv.circle(rgb, (int(x), int(y)), int(radius), (0, 255, 0), 2)
        if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        rv = cv.waitKey(33)
        if rv == ord('a'):#print('left')
            pos_obj[0] += step
            obj.setPosition(pos_obj)
        elif rv==ord('d'):#print('right')
            pos_obj[0] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('w'):#print('up')
            pos_obj[2] += step
            obj.setPosition(pos_obj)
        elif rv==ord('s'):#print('down')
            pos_obj[2] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('f'):#print('front')
            pos_obj[1] += step
            obj.setPosition(pos_obj)
        elif rv==ord('b'): #print('back')
            pos_obj[1] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('q'):
            print('Stop')
            break
        elif rv==ord('c'): #close
            print('closeGripper')
            S.closeGripper("L_gripper")
        elif rv==ord('o'): #open
            print('openGripper')
            S.openGripper("L_gripper")
         
C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()

KeyboardInterrupt: 

In [4]:
C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()

# Visual Servoing with P controller

In [5]:
RealWorld = ry.Config()
RealWorld.addFile("../scenarios/challenge.g")
obj = RealWorld.addFrame("object")
pos_obj = [0,0,.9]
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,1,0])
d=0.1
obj.setShape(ry.ST.ssBox, size=[d,d,d,d])
obj.setColor([1,0,0])
obj.setContact(1)
#obj.setColor([0,0,1])
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")
camera = RealWorld.frame("camera")

C = ry.Config()
C.addFile('../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

In [6]:
eye_in_hand = True #False for eye_to_hand

In [7]:
tau = .01
t = 0
step = 0.05
low_threshold = np.array([30,150,50])
upp_threshold = np.array([255,255,180])
camera.setQuaternion([1,1,0,0])
while True:
    time.sleep(0.01)
    S.step([], tau, ry.ControlMode.none)
    t+=1
    pose = np.zeros(7)
    [y_pos,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])
    pose[:3] = y_pos
    [y,J] = RealWorld.evalFeature(ry.FS.quaternion, ["L_gripperCenter"])
    pose[3:] = y
    if eye_in_hand:
        camera.setPosition(y_pos)
        #camera.setQuaternion(y)
    if t%5 == 0:
        [rgb, depth] = S.getImageAndDepth()
        binary = cv.inRange(cv.cvtColor(rgb, cv.COLOR_BGR2HSV), low_threshold, upp_threshold) #'hue','saturation','value'.
        contours, hier = cv.findContours(binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            (x, y), radius = cv.minEnclosingCircle(contours[0])
            for c in contours:
                (x1, y1), r = cv.minEnclosingCircle(c)
                if r>radius:
                    (x,y)=(x1, y1)
                    radius = r
            cv.circle(rgb, (int(x), int(y)), int(radius), (0, 255, 0), 2)
            f = 0.895
            f = f * 360.                                       # Focal length in pixel
            diameter_real = d * 360                    # 0.1     in pixels 
            k=0.02 #gains
            
            Z = (f*(diameter_real/2.0))/radius      #circle depth            
            xy_target = np.array([x,Z,y]) #my corners
            xy_desired = np.array([640/2, 200.0, 360/2]) #center of the image
            err = xy_desired - xy_target            #error in camera frame
            
            [ee_pos,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])

            vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(ee_pos.shape[0])) @ [-k*err[0], -k*err[1], k*err[2]]
            S.step(vel, tau, ry.ControlMode.velocity)
            #Visual servoing here ----------------------------------------------

        if len(rgb)>0: 
            cv.imshow('OPENCV - rgb', rgb)        
        rv = cv.waitKey(33)
        if rv == ord('a'):#print('left')
            pos_obj[0] += step
            obj.setPosition(pos_obj)
        elif rv==ord('d'):#print('right')
            pos_obj[0] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('w'):#print('up')
            pos_obj[2] += step
            obj.setPosition(pos_obj)
        elif rv==ord('s'):#print('down')
            pos_obj[2] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('f'):#print('front')
            pos_obj[1] += step
            obj.setPosition(pos_obj)
        elif rv==ord('b'): #print('back')
            pos_obj[1] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('q'):
            print('Stop')
            break

C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()

KeyboardInterrupt: 

In [8]:
C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()

# Visual servoing with Image Jacobian

In [9]:
RealWorld = ry.Config()
RealWorld.addFile("../scenarios/challenge.g")
obj = RealWorld.addFrame("object")
pos_obj = [0,0,.9]
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,1,0])
d=0.1
obj.setShape(ry.ST.ssBox, size=[d,d,d,d])
obj.setColor([1,0,0])
obj.setContact(1)
#obj.setColor([0,0,1])

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")
camera = RealWorld.frame("camera")
C = ry.Config()
C.addFile('../scenarios/pandasTable.g')

V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

In [10]:
eye_in_hand = True #False for eye_to_hand

In [11]:
tau = .01
t = 0
step = 0.05
low_threshold = np.array([30,150,50])
upp_threshold = np.array([255,255,180])
camera.setQuaternion([1,1,0,0])
while True:
    time.sleep(0.01)
    S.step([], tau, ry.ControlMode.none)
    t+=1
    pose = np.zeros(7)
    [y_pos,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])
    pose[:3] = y_pos
    [y,J] = RealWorld.evalFeature(ry.FS.quaternion, ["L_gripperCenter"])
    pose[3:] = y
    if eye_in_hand:
        camera.setPosition(y_pos)
        #camera.setQuaternion(y)
        
    if t%5 == 0:
        [rgb, depth] = S.getImageAndDepth()
        binary = cv.inRange(cv.cvtColor(rgb, cv.COLOR_BGR2HSV), low_threshold, upp_threshold) #'hue','saturation','value'.
        contours, hier = cv.findContours(binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            (x, y), radius = cv.minEnclosingCircle(contours[0])
            for c in contours:
                (x1, y1), r = cv.minEnclosingCircle(c)
                if r>radius:
                    (x,y)=(x1, y1)
                    radius = r
            cv.circle(rgb, (int(x), int(y)), int(radius), (0, 255, 0), 2)
            f = 0.895
            f = f * 360.                                       # Focal length in pixel
            diameter_real = d * 360                    # 0.1     in pixels 
            k=0.03 #gains
            
            Z = (f*(diameter_real/2.0))/radius      #circle depth            
            xy_target = np.array([x,y,radius]) #my corners
            xy_desired = np.array([640/2, 360/2, 45.0]) #center of the image
            err = xy_target - xy_desired             #error in camera frame

            Jv = getImageJacobianCFToFF(xy_target[0], xy_target[1], Z, f, diameter_real)
            vel_J = np.dot(np.linalg.inv(Jv),err)
            [ee_pos,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])
            
            vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(ee_pos.shape[0])) @ [-k*vel_J[0], k*vel_J[2], k*vel_J[1]]
            S.step(vel, tau, ry.ControlMode.velocity)
            #Visual servoing here ----------------------------------------------
        
        if len(rgb)>0: 
            cv.imshow('OPENCV - rgb', rgb)        
        rv = cv.waitKey(33)
        if rv == ord('a'):#print('left')
            pos_obj[0] += step
            obj.setPosition(pos_obj)
        elif rv==ord('d'):#print('right')
            pos_obj[0] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('w'):#print('up')
            pos_obj[2] += step
            obj.setPosition(pos_obj)
        elif rv==ord('s'):#print('down')
            pos_obj[2] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('f'):#print('front')
            pos_obj[1] += step
            obj.setPosition(pos_obj)
        elif rv==ord('b'): #print('back')
            pos_obj[1] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('q'):
            print('Stop')
            break
C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()

KeyboardInterrupt: 

In [6]:
def getImageJacobianCFToFF(u, v, z, f, diameter):
    Jv = np.zeros((3,3))
    
    Jv[0][0] = -f/z
    Jv[0][1] = 0
    Jv[0][2] = u/z
    
    Jv[1][0] = 0
    Jv[1][1] = -f/z
    Jv[1][2] = v/z

    Jv[2][0] = 0
    Jv[2][1] = 0
    Jv[2][2] = -(diameter*f)/(z*z)

    return Jv

In [13]:
C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()

# Visual servoing with red object

In [14]:
RealWorld = ry.Config()
RealWorld.addFile("../scenarios/challenge.g")
obj = RealWorld.addFrame("object")
pos_obj = [0,0,.9]
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,1,0])
d=0.1
obj.setShape(ry.ST.ssBox, size=[d,d,d,d])
obj.setColor([1,0,0])
obj.setContact(1)
#obj.setColor([0,0,1])

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")
camera = RealWorld.frame("camera")
C = ry.Config()
C.addFile('../scenarios/pandasTable.g')

V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

In [15]:
eye_in_hand = True

In [17]:
import time
from collections import deque
import imutils

tau = .01
t = 0
step = 0.05
camera.setQuaternion([1,1,0,0])
low_threshold = np.array([30,150,50])
upp_threshold = np.array([255,255,180])
cap = cv.VideoCapture(0)
time.sleep(1.0)
pts = deque(maxlen=10)
vs = cap

while True:
    time.sleep(0.01)
    S.step([], tau, ry.ControlMode.none)
    t+=1
    pose = np.zeros(7)
    [y_pos,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])
    pose[:3] = y_pos
    [y,J] = RealWorld.evalFeature(ry.FS.quaternion, ["L_gripperCenter"])
    pose[3:] = y
    if eye_in_hand:
        camera.setPosition(y_pos)
        #camera.setQuaternion(y)
    if t%5 == 0:
        [rgb, depth] = S.getImageAndDepth()
        
        
        ret, frame = vs.read()
        if frame is not None:
            blurred = cv.GaussianBlur(frame, (11, 11), 0)
            hsv = cv.cvtColor(blurred, cv.COLOR_BGR2HSV)

            mask = cv.inRange(hsv, low_threshold, upp_threshold)
            mask = cv.erode(mask, None, iterations=2) #Erosion
            mask = cv.dilate(mask, None, iterations=2)
    
            
            cnts = cv.findContours(mask.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            center = None
            
            if len(cnts) > 0:
                c = max(cnts, key=cv.contourArea)
                ((x, y), radius) = cv.minEnclosingCircle(c)
                M = cv.moments(c)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                if radius > 10:
                    cv.circle(frame, (int(x), int(y)), int(radius),
                        (0, 255, 255), 2)
                    cv.circle(frame, center, 5, (0, 0, 255), -1)
                    (x1, y1) = x, y

            pts.appendleft(center)

            for i in range(1, len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    continue

                thickness = int(np.sqrt(6 / float(i + 1)) * 2.5)
                if thickness > 0:
                    cv.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)            

            f = 0.895
            f = f * 360.                                       # Focal length in pixel
            diameter_real = d * 360                    # 0.1     in pixels 
            k=0.02 #gains

            Z = (f*(diameter_real/2.0))/radius      #circle depth            
            xy_target = np.array([x1,Z,y1]) #my corners
            xy_desired = np.array([640/2, 200.0, 360/2]) #center of the image
            err = xy_desired - xy_target            #error in camera frame
            [ee_pos,J] = RealWorld.evalFeature(ry.FS.position, ["L_gripperCenter"])

            vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(ee_pos.shape[0])) @ [-k*err[0], -k*err[1], k*err[2]]
            S.step(vel, tau, ry.ControlMode.velocity)
            #Visual servoing here ----------------------------------------------

        if len(rgb)>0: 
            cv.imshow('OPENCV - rgb', rgb) 
            cv.imshow('frame',frame)
        rv = cv.waitKey(33)
        if rv == ord('a'):#print('left')
            pos_obj[0] += step
            obj.setPosition(pos_obj)
        elif rv==ord('d'):#print('right')
            pos_obj[0] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('w'):#print('up')
            pos_obj[2] += step
            obj.setPosition(pos_obj)
        elif rv==ord('s'):#print('down')
            pos_obj[2] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('f'):#print('front')
            pos_obj[1] += step
            obj.setPosition(pos_obj)
        elif rv==ord('b'): #print('back')
            pos_obj[1] -= step
            obj.setPosition(pos_obj)
        elif rv==ord('q'):
            print('Stop')
            break

C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()
cap.release()

KeyboardInterrupt: 

In [18]:
C = 0
RealWorld = 0
S = 0
V=0
cv.destroyAllWindows()
cap.release()

# Exercise 1:


In [22]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0)
low_threshold = np.array([30,150,50])
upp_threshold = np.array([255,255,180])

while True:
    ret, img = cap.read()
    if img is None:
        #break
        print('Frame is none')
        continue
   
    binary = cv.inRange(cv.cvtColor(img, cv.COLOR_BGR2HSV), low_threshold, upp_threshold) #'hue','saturation','value'.
    #ret, binary = cv.threshold(cv.cvtColor(img, cv.COLOR_BGR2GRAY),
     #               127, 255, cv.THRESH_BINARY)
    
    contours, hier = cv.findContours(binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #-----------------------------------------------------------------------
    for c in contours:
        x, y, w, h = cv.boundingRect(c)
        cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

        (x, y), radius = cv.minEnclosingCircle(c)
        center = (int(x), int(y))
        radius = int(radius)
        img = cv.circle(img, center, radius, (255, 0, 0), 2)

    cv.drawContours(img, contours, -1, (255, 255, 0), 1)
    cv.imshow("img withcontours", img)
    cv.imshow('binary',binary)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()

KeyboardInterrupt: 

In [23]:
cap.release()
cv.destroyAllWindows()

# Exercise 2 

In [24]:
import numpy as np
import cv2 as cv
import time

cap = cv.VideoCapture(0)
ret, first = cap.read()

first_gray = cv.cvtColor(first, cv.COLOR_BGR2GRAY)
first_gray = cv.GaussianBlur(first_gray, (21, 21), 0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (21, 21), 0)

    difference = cv.absdiff(gray, first_gray)

    thresh = cv.threshold(difference, 25, 255, cv.THRESH_BINARY)[1]
    thresh = cv.dilate(thresh, None, iterations=2)
    
    contours, hier = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    if len(contours) != 0:
        cv.drawContours(frame, contours, -1, 255, 3)
        c = max(contours, key = cv.contourArea)
        x,y,w,h = cv.boundingRect(c)
        cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    cv.imshow('frame',frame)
    cv.imshow("thresh", thresh)
    key = cv.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cap.release()
cv.destroyAllWindows()